In [2]:
import pandas as pd
import plotly.express as px

# Load and clean (assume you already loaded df, otherwise do it again)

# Normalize Clinics
df['Clinics_Score'] = df['No__of_abortion_clinics_2020'] / df['No__of_abortion_clinics_2020'].max()

# Normalize Funding
df['Funding_Score'] = (df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] * 1000) / (df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'].max() * 1000)

# Combine into Raw Resource Score
df['Raw_Resource_Score'] = df['Clinics_Score'] + df['Funding_Score']

# Adjusted Resource Score per Woman (normalize by population)
df['Adjusted_Resource_Score'] = df['Raw_Resource_Score'] / df['Pop_15_44']

# ------------------
# PLOTS
# ------------------

# Raw Resource Strength (no population adjustment)
fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Raw_Resource_Score',
    scope='usa',
    color_continuous_scale="Blues",
    title='Raw Combined Resource Score (Clinics + Funding)'
)
fig.update_layout(template="plotly_white")
fig.show()

# Adjusted Resource Strength (per population)
fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Adjusted_Resource_Score',
    scope='usa',
    color_continuous_scale="Greens",
    title='Combined Resource Score Adjusted for Population'
)
fig.update_layout(template="plotly_white")
fig.show()


In [13]:
import pandas as pd
import plotly.express as px

# Load dataset
df = pd.read_csv("GuttmacherInstituteAbortionDataByState.csv")

# Clean columns
df.columns = df.columns.str.strip().str.replace('–', '-').str.replace('—', '-').str.replace(' ', '_') \
    .str.replace(',', '').str.replace('(', '').str.replace(')', '').str.replace('’', '').str.replace('.', '_')

df.rename(columns={'U.S._State': 'U_S__State'}, inplace=True)

# Convert important columns
df['No__of_abortion_clinics_2020'] = pd.to_numeric(df['No__of_abortion_clinics_2020'], errors='coerce')
df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] = pd.to_numeric(
    df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'], errors='coerce')
df['No__of_abortions_by_state_of_occurrence_2020'] = pd.to_numeric(
    df['No__of_abortions_by_state_of_occurrence_2020'], errors='coerce')
df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020'] = pd.to_numeric(
    df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020'], errors='coerce')

# Estimate Population of Women 15–44
df['Pop_15_44'] = (df['No__of_abortions_by_state_of_occurrence_2020'] * 1000) / df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020']

# Normalize Clinics and Funding
df['Clinics_Score'] = df['No__of_abortion_clinics_2020'] / df['No__of_abortion_clinics_2020'].max()
df['Funding_Score'] = (df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] * 1000) / (df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'].max() * 1000)

# Combined Resource Score
df['Raw_Resource_Score'] = df['Clinics_Score'] + df['Funding_Score']
df['Adjusted_Resource_Score'] = df['Raw_Resource_Score'] / df['Pop_15_44']

# Add State Codes
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
df['State_Code'] = df['U_S__State'].map(state_abbreviations)
df = df.dropna(subset=['State_Code'])

# --------------------
# Plot 1: Raw Resource Score (continuous blues)
# --------------------

fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Raw_Resource_Score',
    color_continuous_scale=px.colors.sequential.Blues,
    scope='usa',
    title='Raw Combined Resource Score (Normalized Clinics + Funding)'
)
fig.update_layout(template="plotly_white", coloraxis_colorbar_title="Resource Strength (Raw)")
fig.show()

# --------------------
# Plot 2: Adjusted Resource Score (binned and blues)
# --------------------

# Bin Adjusted Score
df['Adjusted_Bin_Label'] = pd.qcut(
    df['Adjusted_Resource_Score'],
    q=5,
    labels=["Bottom 20%", "20–40%", "40–60%", "60–80%", "Top 20%"]
)

color_scale_discrete = ['#deebf7', '#9ecae1', '#6baed6', '#3182bd', '#08519c']  # nice Blues from light to dark

fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Adjusted_Bin_Label',
    color_discrete_sequence=color_scale_discrete,
    scope='usa',
    title='Population Adjusted Resource Score (Normalized Clinics + Funding)'
)
fig.update_layout(template="plotly_white", legend_title="Resource Strength Percentile")
fig.show()


In [15]:
import pandas as pd
import plotly.express as px

# Load dataset
df = pd.read_csv("GuttmacherInstituteAbortionDataByState.csv")

# Clean columns
df.columns = df.columns.str.strip().str.replace('–', '-').str.replace('—', '-').str.replace(' ', '_') \
    .str.replace(',', '').str.replace('(', '').str.replace(')', '').str.replace('’', '').str.replace('.', '_')

df.rename(columns={'U.S._State': 'U_S__State'}, inplace=True)

# Convert important columns
df['No__of_abortion_clinics_2020'] = pd.to_numeric(df['No__of_abortion_clinics_2020'], errors='coerce')
df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] = pd.to_numeric(
    df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'], errors='coerce')
df['No__of_abortions_by_state_of_occurrence_2020'] = pd.to_numeric(
    df['No__of_abortions_by_state_of_occurrence_2020'], errors='coerce')
df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020'] = pd.to_numeric(
    df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020'], errors='coerce')

# Estimate Population of Women 15–44
df['Pop_15_44'] = (df['No__of_abortions_by_state_of_occurrence_2020'] * 1000) / df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020']

# Normalize Clinics and Funding
df['Clinics_Score'] = df['No__of_abortion_clinics_2020'] / df['No__of_abortion_clinics_2020'].max()
df['Funding_Score'] = (df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] * 1000) / (df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'].max() * 1000)

# Combined Resource Score
df['Raw_Resource_Score'] = df['Clinics_Score'] + df['Funding_Score']
df['Adjusted_Resource_Score'] = df['Raw_Resource_Score'] / df['Pop_15_44']

# Add State Codes
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
df['State_Code'] = df['U_S__State'].map(state_abbreviations)
df = df.dropna(subset=['State_Code'])

# --------------------
# Plot 1: Raw Resource Score (continuous blues)
# --------------------

fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Raw_Resource_Score',
    color_continuous_scale=px.colors.sequential.Blues,
    scope='usa',
    title='Raw Combined Resource Score (Normalized Clinics + Funding)'
)
fig.update_layout(template="plotly_white", coloraxis_colorbar_title="Resource Strength (Raw)")
fig.show()

# --------------------
# Plot 2: Adjusted Resource Score (binned and blues)
# --------------------

# Bin Adjusted Score
df['Adjusted_Bin_Label'] = pd.qcut(
    df['Adjusted_Resource_Score'],
    q=5,
    labels=["Bottom 20%", "20–40%", "40–60%", "60–80%", "Top 20%"]
)

color_scale_discrete = ['#deebf7', '#9ecae1', '#6baed6', '#3182bd', '#08519c']  # nice Blues from light to dark

fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Adjusted_Bin_Label',
    color_discrete_sequence=color_scale_discrete,
    scope='usa',
    title='Population Adjusted Resource Score (Normalized Clinics + Funding)'
)
fig.update_layout(template="plotly_white", legend_title="Resource Strength Percentile")
fig.show()


In [18]:
# Plot 1: Raw Score (Unadjusted)
fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Raw_Resource_Score',
    color_continuous_scale=px.colors.sequential.Blues,
    scope='usa',
    title='California is Unmatched in Abortion Resources<br><sup>Total Clinics and State Abortion Funding — Without Adjusting for Population or Geography</sup>'
)
fig.update_layout(
    template="plotly_white",
    coloraxis_colorbar_title="Total Resource Score<br>(Clinics + Funding, Normalized)"
)
fig.show()

# Plot 2: Density-Adjusted Score (Explicit Description)
fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Density_Bin_Label',
    color_discrete_sequence=color_scale_discrete,
    scope='usa',
    title='Why the East Coast Dominates in Abortion Access<br><sup>Based on Clinics and State Funding per Capita and per Square Mile</sup>'
)
fig.update_layout(
    template="plotly_white",
    legend_title="Percentile Based on:<br>Clinics + Funding / (Population × Land Area)"
)
fig.show()

